In [12]:
import gzip
import pandas as pd
import os


#pd.set_option('display.max_columns', None)  
#pd.set_option('display.width', 200)         
#pd.set_option('display.max_rows', 10)       

archivo_gz = 'GRCh38.nr_insertions.common.acmg_genes.tsv.gz'

archivo_salida_tsv = 'archivo_descomprimido.tsv'

with gzip.open(archivo_gz, 'rt') as f:
    df = pd.read_csv(f, sep='\t', quotechar='"', escapechar='\\',header=1)


print(df)
print(df.columns)

df.to_csv(archivo_salida_tsv, sep='\t', index=False)


    #chr  outermost_start  outermost_stop  variant_count     variant_type  \
0      1         17028677        17028677              1    alu_insertion   
1      1         45338416        45338416              1    alu_insertion   
2      1         68434395        68434395              1    alu_insertion   
3      1        115710237       115710237              1    alu_insertion   
4      1        115761484       115761484              1        insertion   
..   ...              ...             ...            ...              ...   
256   22         29611368        29611368              1        insertion   
257   22         29671063        29671063              1    alu_insertion   
258   22         29676948        29676948              2        insertion   
259   22         29696474        29696474              1    alu_insertion   
260    X         38381635        38381635              1  line1_insertion   

         method                  analysis                platform  \
0     

In [13]:
# Verificar si las columnas específicas están completamente llenas de valores nulos
columnas_con_nulos = [col for col in ['platform', 'clinical_significance', 'clinvar_accession' ] if df[col].isnull().all()]

# Imprimir las columnas que están completamente llenas de nulos
print("Columnas completamente llenas de valores nulos:", columnas_con_nulos)

# Si decides eliminar estas columnas específicas, puedes hacerlo así:
df = df.drop(columns=columnas_con_nulos)

# Verificar el resultado
print(df.head())

Columnas completamente llenas de valores nulos: ['clinical_significance', 'clinvar_accession']
  #chr  outermost_start  outermost_stop  variant_count   variant_type  \
0    1         17028677        17028677              1  alu_insertion   
1    1         45338416        45338416              1  alu_insertion   
2    1         68434395        68434395              1  alu_insertion   
3    1        115710237       115710237              1  alu_insertion   
4    1        115761484       115761484              1      insertion   

       method            analysis                platform  \
0     Merging             Merging  See merged experiments   
1  Sequencing               Other                     NaN   
2  Sequencing               Other                     NaN   
3  Sequencing               Other                     NaN   
4  Sequencing  Sequence_alignment                     NaN   

                                           study       variant bin_size  \
0  1000_Genomes_Consorti

In [14]:
columnas_con_algun_nulo = df.columns[df.isnull().any()].tolist()

print("Columnas que serán eliminadas porque contienen al menos un valor nulo:", columnas_con_algun_nulo)

# Eliminar las columnas que contienen al menos un valor nulo
df = df.drop(columns=columnas_con_algun_nulo)

# Verificar el resultado
print(df.head())

Columnas que serán eliminadas porque contienen al menos un valor nulo: ['platform']
  #chr  outermost_start  outermost_stop  variant_count   variant_type  \
0    1         17028677        17028677              1  alu_insertion   
1    1         45338416        45338416              1  alu_insertion   
2    1         68434395        68434395              1  alu_insertion   
3    1        115710237       115710237              1  alu_insertion   
4    1        115761484       115761484              1      insertion   

       method            analysis  \
0     Merging             Merging   
1  Sequencing               Other   
2  Sequencing               Other   
3  Sequencing               Other   
4  Sequencing  Sequence_alignment   

                                           study       variant bin_size  \
0  1000_Genomes_Consortium_Phase_3_SV_Submission  essv16984100    small   
1                     gnomAD_Structural_Variants  nssv16024648    small   
2                     gnomAD_

In [15]:
#Para mi prueba solo quiero las columnas de chr, start y end

df = df[['#chr', 'outermost_start', 'outermost_stop', 'method', 'variant_type', 'variant', 'gene']]
df['variant'] = df['variant'].str.split(';')
df = df.explode('variant')
# Verificar el resultado
print(df.head())


  #chr  outermost_start  outermost_stop      method   variant_type  \
0    1         17028677        17028677     Merging  alu_insertion   
1    1         45338416        45338416  Sequencing  alu_insertion   
2    1         68434395        68434395  Sequencing  alu_insertion   
3    1        115710237       115710237  Sequencing  alu_insertion   
4    1        115761484       115761484  Sequencing      insertion   

        variant   gene  
0  essv16984100   SDHB  
1  nssv16024648  MUTYH  
2  nssv16025035  RPE65  
3  nssv16026918  CASQ2  
4  nssv15762567  CASQ2  


In [16]:
df.rename(columns={'#chr': 'CHROM', 'outermost_start': 'POS', 'outermost_stop': 'END'}, inplace=True)
print(df.head())

  CHROM        POS        END      method   variant_type       variant   gene
0     1   17028677   17028677     Merging  alu_insertion  essv16984100   SDHB
1     1   45338416   45338416  Sequencing  alu_insertion  nssv16024648  MUTYH
2     1   68434395   68434395  Sequencing  alu_insertion  nssv16025035  RPE65
3     1  115710237  115710237  Sequencing  alu_insertion  nssv16026918  CASQ2
4     1  115761484  115761484  Sequencing      insertion  nssv15762567  CASQ2


In [17]:
specific_row = df.iloc[1]
print(specific_row)

CHROM                       1
POS                  45338416
END                  45338416
method             Sequencing
variant_type    alu_insertion
variant          nssv16024648
gene                    MUTYH
Name: 1, dtype: object


In [18]:
# Añadir nuevas columnas que voy a necesitar
df['DB'] = 'dbVar' # Columna base de datos

In [19]:
specific_row_2 = df.iloc[1]
print(specific_row_2)

CHROM                       1
POS                  45338416
END                  45338416
method             Sequencing
variant_type    alu_insertion
variant          nssv16024648
gene                    MUTYH
DB                      dbVar
Name: 1, dtype: object


In [20]:
# Define la ruta y el nombre del archivo TSV de salida
archivo_salida_tsv = 'salida_GRCh38.nr_insertion.tsv'
carpeta_salida = 'Salidas'
archivo_salida_tsv = os.path.join(carpeta_salida, archivo_salida_tsv)
# Guardar el DataFrame en un archivo TSV
df.to_csv(archivo_salida_tsv, sep='\t', index=False)

print(f"DataFrame guardado con éxito en '{carpeta_salida}'")


DataFrame guardado con éxito en 'Salidas'


In [21]:
#archivo_gz = 'GRCh38.variant_call.clinical.conflicting_or_other.vcf.gz'

# Descomprimir y leer el archivo
#with gzip.open(archivo_gz, 'rt') as archivo:
#    for linea in archivo:
#        print(linea)

In [22]:
# Lista de columnas relevantes según tu YARRRML
# Ajusta esta lista según las columnas que realmente estén definidas en tu YARRRML
relevant_columns = ['#chr', 'outermost_start', 'outermost_stop', 'variant']

# Verifica si todas las columnas relevantes existen en el DataFrame, para evitar errores
missing_columns = [col for col in relevant_columns if col not in df.columns]
if missing_columns:
    print("Advertencia: Las siguientes columnas relevantes no se encontraron en el archivo:", missing_columns)
else:
    # Elimina las columnas que no están en 'relevant_columns'
    df = df[relevant_columns]

    # Guarda el DataFrame resultante en un nuevo archivo
    output_path = 'archivo_filtrado.tsv'  # Asegúrate de actualizar esta ruta
    df.to_csv(output_path, sep='\t', index=False)

    print(f"Archivo guardado exitosamente en: {output_path}")

Advertencia: Las siguientes columnas relevantes no se encontraron en el archivo: ['#chr', 'outermost_start', 'outermost_stop']
